In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()   

PROFILE_AWS = os.getenv("PROFILE_AWS")

ia-temp


In [55]:
import json
import boto3

# Crear una sesión con el perfil
sesion_aws = boto3.Session(profile_name=PROFILE_AWS, region_name=os.getenv("region_name"))

bedrock = sesion_aws.client("bedrock-runtime")
s3vectors = sesion_aws.client("s3vectors")


In [ ]:

# Query vector index.
response = s3vectors.list_vectors(
    vectorBucketName="media-embeddings",
    indexName="tets-docs",
)
print(json.dumps(response["vectors"], indent=2))

In [3]:
keys = [item["key"] for item in response["vectors"] if "key" in item]
keys


[]

In [ ]:
response = s3vectors.delete_vectors(
    vectorBucketName='media-embeddings',
    indexName='tets-docs',
    #indexArn='string',
    keys=keys
)

response

In [9]:
!aws s3vectors delete-vector-bucket \
  --vector-bucket-name "media-embeddings" --profile ia-temp


An error occurred (ConflictException) when calling the DeleteVectorBucket operation: The specified vector bucket is not empty


In [12]:
!aws s3vectors list-vectors --profile ia-temp\
  --vector-bucket-name "media-embeddings" \
  --index-name "movies" \
  --segment-count 2 \
  --segment-index 0 \
  --return-data \
  --return-metadata

{
    "vectors": []
}


In [13]:
!aws s3vectors delete-index --profile ia-temp --vector-bucket-name "media-embeddings" \
          --index-name "movies"

In [14]:
!aws s3vectors delete-index --profile ia-temp --vector-bucket-name "media-embeddings" \
          --index-name "tets-docs"

In [15]:
!aws s3vectors delete-vector-bucket \
  --vector-bucket-name "media-embeddings" --profile ia-temp

## Eliminar todo con python 
[Guia Eliminar vector buckets](https://docs.aws.amazon.com/AmazonS3/latest/userguide/s3-vectors-buckets-delete.html)

[Guia s3vectors python](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3vectors.html)

### Listar los buckets

In [25]:
resp = s3vectors.list_vector_buckets()
vect_buckets = [item["vectorBucketName"] for item in resp["vectorBuckets"]]
vect_buckets

['bedrock-knowledge-base-71yquh',
 'bedrock-knowledge-base-s5k766',
 'bedrock-knowledge-base-u5ku3r',
 's3vector-demo-01']

### Listar los indices de los buckets (uno)

In [37]:
response = s3vectors.list_indexes(
    vectorBucketName=vect_buckets[0]
)

indexes = [item["indexName"] for item in response['indexes']]
{"s3vectorBucket":vect_buckets[0],"indexes":indexes}

{'s3vectorBucket': 'bedrock-knowledge-base-71yquh',
 'indexes': ['bedrock-knowledge-base-default-index']}

### Todos los s3 vector buckets detectados y sus indices como diccionario

In [38]:
ls=[]
for bucket in vect_buckets:
    response = s3vectors.list_indexes(
        vectorBucketName=bucket
    )

    indexes = [item["indexName"] for item in response['indexes']]
    ls.append({"s3vectorBucket":bucket,"indexes":indexes})
ls

[{'s3vectorBucket': 'bedrock-knowledge-base-71yquh',
  'indexes': ['bedrock-knowledge-base-default-index']},
 {'s3vectorBucket': 'bedrock-knowledge-base-s5k766',
  'indexes': ['bedrock-knowledge-base-default-index']},
 {'s3vectorBucket': 'bedrock-knowledge-base-u5ku3r',
  'indexes': ['bedrock-knowledge-base-default-index']},
 {'s3vectorBucket': 's3vector-demo-01', 'indexes': ['s3vector-index-kb01']}]

In [ ]:
for dc in ls:
    print(f"index: {dc["s3vectorBucket"]}")
    for index in dc["indexes"]:
        
        print(f"index: {index}")
        response = s3vectors.list_vectors(
            vectorBucketName=dc["s3vectorBucket"],
            indexName=index,
        )
        print(json.dumps(response["vectors"], indent=2))
        keys = [item["key"] for item in response["vectors"] if "key" in item]
        keys

index: bedrock-knowledge-base-71yquh
index: bedrock-knowledge-base-default-index
[]
index: bedrock-knowledge-base-s5k766
index: bedrock-knowledge-base-default-index
[]
index: bedrock-knowledge-base-u5ku3r
index: bedrock-knowledge-base-default-index
[]
index: s3vector-demo-01
index: s3vector-index-kb01
[]


In [ ]:
for dc in ls:
    print(f"index: {dc["s3vectorBucket"]}")
    for index in dc["indexes"]:
        
        print(f"index: {index}")
        response = s3vectors.list_vectors(
            vectorBucketName=dc["s3vectorBucket"],
            indexName=index,
        )
        print(json.dumps(response["vectors"], indent=2))
        
        keys = [item["key"] for item in response["vectors"] if "key" in item]
        s3vectors.delete_vectors(
            vectorBucketName=dc["s3vectorBucket"],
            indexName=index,
            keys=keys
        )
        print(f"Deleted vectors in index: {index}")

In [42]:
for dc in ls:
    print(f"index: {dc["s3vectorBucket"]}")
    for index in dc["indexes"]:
        s3vectors.delete_index(
            vectorBucketName=dc["s3vectorBucket"],
            indexName=index
        )
        print(f"Deleted index: {index}")
    s3vectors.delete_vector_bucket(
        vectorBucketName=dc["s3vectorBucket"]
    )
    print(f"Deleted bucket: {dc["s3vectorBucket"]}")

index: bedrock-knowledge-base-71yquh
Deleted index: bedrock-knowledge-base-default-index
Deleted bucket: bedrock-knowledge-base-71yquh
index: bedrock-knowledge-base-s5k766
Deleted index: bedrock-knowledge-base-default-index
Deleted bucket: bedrock-knowledge-base-s5k766
index: bedrock-knowledge-base-u5ku3r
Deleted index: bedrock-knowledge-base-default-index
Deleted bucket: bedrock-knowledge-base-u5ku3r
index: s3vector-demo-01
Deleted index: s3vector-index-kb01
Deleted bucket: s3vector-demo-01


### Validar el purgado total

In [43]:
resp = s3vectors.list_vector_buckets()
vect_buckets = [item["vectorBucketName"] for item in resp["vectorBuckets"]]
vect_buckets

[]